In [1]:
import tensorflow as tf


In [2]:
tf.__version__

'2.1.0'

In [3]:
import keras_preprocessing
train_dir='data/train'
test_dir='data/test'
train_data_generator=keras_preprocessing.image.ImageDataGenerator(rescale=1.0/255)
test_data_generator=keras_preprocessing.image.ImageDataGenerator(rescale=1.0/255)
train_data=train_data_generator.flow_from_directory(
        train_dir,
        target_size=(256,256),
        class_mode='categorical'
)
test_data=test_data_generator.flow_from_directory(
        test_dir,
        target_size=(256,256),
        class_mode='categorical'
)

Found 34934 images belonging to 15 classes.
Found 3831 images belonging to 15 classes.


In [7]:
import tensorflow.keras as keras

In [22]:
model=keras.models.Sequential([
    keras.layers.Conv2D(4,(3,3),activation='relu',input_shape=(256,256,3)),
    keras.layers.MaxPooling2D(3,3),
    keras.layers.Conv2D(8,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(3,3),
    #keras.layers.Conv2D(64,(3,3),activation='relu'),
    #keras.layers.MaxPooling2D(3,3),
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(15,activation='softmax')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 254, 254, 4)       112       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 82, 82, 8)         296       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 27, 27, 8)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 5832)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                373312    
_________________________________________________________________
dense_11 (Dense)             (None, 15)               

In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
history=model.fit(train_data,epochs=20,validation_data=test_data,verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 582 steps, validate for 64 steps
Epoch 1/20
582/582 [==============================] - 195s 335ms/step - loss: 1.6664 - accuracy: 0.4721 - val_loss: 1.1489 - val_accuracy: 0.6262
Epoch 2/20
582/582 [==============================] - 68s 117ms/step - loss: 0.9911 - accuracy: 0.6773 - val_loss: 0.8767 - val_accuracy: 0.7068
Epoch 3/20
582/582 [==============================] - 68s 117ms/step - loss: 0.7799 - accuracy: 0.7411 - val_loss: 0.7967 - val_accuracy: 0.7339
Epoch 4/20
582/582 [==============================] - 70s 120ms/step - loss: 0.6580 - accuracy: 0.7802 - val_loss: 0.7336 - val_accuracy: 0.7447
Epoch 5/20
582/582 [==============================] - 71s 121ms/step - loss: 0.5761 - accuracy: 0.8094 - val_loss: 0.6941 - val_accuracy: 0.7683
Epoch 6/20
582/582 [==============================] - 79s 136ms/step - loss: 0.5044 - accuracy: 0.8344 - val_loss: 0.6430 - val_accuracy: 0.7850
Epoch 7/20
582/582 [================

KeyboardInterrupt: 

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1491219390589249635
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3140970086
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9194218378274989017
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [6]:
print(tf.test.is_built_with_cuda())

True


In [10]:
def build(input_size,output_size):
    model=keras.models.Sequential()
    
    no_of_conv_layers=2
    no_of_dense_layers=2
    no_of_filters=4
    no_of_dense_neurons=64
    size_of_filter=3
    model.add(keras.layers.Conv2D(4,(3,3),activation='relu',input_shape=(256,256,3)))
    model.add(keras.layers.MaxPooling2D(size_of_filter,size_of_filter))
    for i in range(no_of_conv_layers):
        no_of_filters*=2
        model.add(keras.layers.Conv2D(no_of_filters,(size_of_filter,size_of_filter),activation='relu'))
        model.add(keras.layers.MaxPooling2D(size_of_filter,size_of_filter))
        
    model.add(keras.layers.Flatten())
    for i in range(no_of_dense_layers):
        model.add(keras.layers.Dense(no_of_dense_neurons,activation='relu'))
        no_of_dense_neurons/=2
    model.add(keras.layers.Dense(output_size,activation='softmax'))
    model.summary()
    
    return model
def try_models(input_size,output_size):
    accuracy_scores=[]
    optimizers=['adam','rmsprop']
    loss=['MSE','categorical_crossentropy']
    for i in optimizers:
        for j in loss:
            print("Testing with "+i+" as optimizer and "+j+" as loss function")
            model=build(input_size,output_size)
            model.compile(optimizer=i,loss=j,metrics=['accuracy'])
            history=model.fit(train_data,epochs=1,validation_data=test_data,verbose=1)
            accuracy_scores.append(history.history['accuracy'])
    print(accuracy_scores)
    return


In [17]:
try_models((256,256,3),15)

Testing with adam as optimizer and MSE as loss function
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 254, 254, 4)       112       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 252, 252, 8)       296       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 84, 84, 8)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 56448)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                3612736   
_________________________________________________________________
dense_12 (Dense)             (None, 15)                975       
Total params: 3,614,119
Trainable params: 3,614,119
Non-trainabl

Hence Using Adam optimizer with Categorical Cross Entropy

In [16]:
class myCallback(tf.keras.callbacks.Callback):
    weighted_train_acc=0
    weighted_val_acc=0
    def on_epoch_end(self, epoch, logs={}): 
        #if(logs.get('acc') > ACCURACY_THRESHOLD):   
        #print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
        if(weighted_train_acc > logs.get('acc')):
            self.model.stop_training = True
        else:
            weighted_train_acc = weighted_train_acc+logs.get('acc')*0.9

mycallback = myCallback()

In [11]:
model=build((256,256,3),15)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],callbacks=[mycallback])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 4)       112       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 82, 82, 8)         296       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 27, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 16)        1168      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)             

In [20]:
history=model.fit(train_data,epochs=5,validation_data=test_data,verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1092 steps, validate for 120 steps
Epoch 1/5
1092/1092 [==============================] - 137s 125ms/step - loss: 0.0800 - accuracy: 0.9707 - val_loss: 0.3352 - val_accuracy: 0.9126
Epoch 2/5
1092/1092 [==============================] - 144s 132ms/step - loss: 0.0825 - accuracy: 0.9711 - val_loss: 0.3938 - val_accuracy: 0.8979
Epoch 3/5
1092/1092 [==============================] - 145s 132ms/step - loss: 0.0805 - accuracy: 0.9720 - val_loss: 0.2946 - val_accuracy: 0.9280
Epoch 4/5
1092/1092 [==============================] - 152s 139ms/step - loss: 0.0680 - accuracy: 0.9764 - val_loss: 0.2864 - val_accuracy: 0.9256
Epoch 5/5
1092/1092 [==============================] - 151s 138ms/step - loss: 0.0682 - accuracy: 0.9759 - val_loss: 0.3123 - val_accuracy: 0.9201


In [21]:
model.save('model.h5')